<a href="https://colab.research.google.com/github/BrennoCMessias/VacumSearch/blob/main/Vacum_p2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
from collections import deque

class VacuumEnvironment:
    def __init__(self, num_rows, num_cols, dirty_rooms):
        self.num_rows = num_rows
        self.num_cols = num_cols
        self.dirty_rooms = set(dirty_rooms)  # Usar um conjunto para dirty_rooms
        self.current_row = random.randint(0, num_rows - 1)
        self.current_col = random.randint(0, num_cols - 1)
        self.history = []

    def is_dirty(self, row, col):
        return (row, col) in self.dirty_rooms

    def clean(self, row, col):
        self.dirty_rooms.remove((row, col))
        self.history.append((row, col, "Limpou"))

    def move_up(self):
        self.current_row = max(0, self.current_row - 1)
        self.history.append((self.current_row, self.current_col, "Moveu"))

    def move_down(self):
        self.current_row = min(self.num_rows - 1, self.current_row + 1)
        self.history.append((self.current_row, self.current_col, "Moveu"))

    def move_left(self):
        self.current_col = max(0, self.current_col - 1)
        self.history.append((self.current_row, self.current_col, "Moveu"))

    def move_right(self):
        self.current_col = min(self.num_cols - 1, self.current_col + 1)
        self.history.append((self.current_row, self.current_col, "Moveu"))

    def percept(self):
        return (self.current_row, self.current_col), self.is_dirty(self.current_row, self.current_col)

def display_environment(environment):
    print("AMBIENTE:")
    for row in range(environment.num_rows):
        for col in range(environment.num_cols):
            if (row, col) == (environment.current_row, environment.current_col):
                print("O", end=" ")
            elif environment.is_dirty(row, col):
                print("X", end=" ")
            else:
                print(".", end=" ")
        print()
    print("\n")

def base_control(environment):

    # Move para a posição (0, 0) se não estiver lá
    while environment.current_row != 0 or environment.current_col != 0:
        if environment.current_row > 0:
            environment.move_up()
        elif environment.current_col > 0:
            environment.move_left()
        display_environment(environment)

    # Analisa as colunas
    for col in range(environment.num_cols):
        while environment.current_row < environment.num_rows - 1:
            # Move para baixo na coluna
            environment.move_down()
            display_environment(environment)

            # Analisa e limpa se a sala estiver suja
            current_position, is_dirty = environment.percept()
            if is_dirty:
                environment.clean(*current_position)
                display_environment(environment)

        while environment.current_row > 0:
            # Move para cima na coluna
            environment.move_up()
            display_environment(environment)

            # Analisa e limpa se a sala estiver suja
            current_position, is_dirty = environment.percept()
            if is_dirty:
                environment.clean(*current_position)
                display_environment(environment)

        # Move para a próxima coluna, se não estiver na última
        if col < environment.num_cols - 1:
            environment.move_right()
            display_environment(environment)

    # Exibir ambiente após a limpeza
    display_environment(environment)

def manual_control(environment):
    while environment.dirty_rooms:
        display_environment(environment)
        current_position, is_dirty = environment.percept()
        status = "Suja" if is_dirty else "Limpa"
        print(f"Aspirador na posição {current_position}. Sala está {status}")

        action = input("Escolha ação (Mover para cima - 'U'. \nMover para baixo - 'D'. \nMover para esquerda - 'L'. \nMover para direita - 'R'. \nLimpar - 'C' \nEsperar - 'ES'): ")
        if action == 'U':
            environment.move_up()
        elif action == 'D':
            environment.move_down()
        elif action == 'L':
            environment.move_left()
        elif action == 'R':
            environment.move_right()
        elif action == 'C':
            if is_dirty:
                environment.clean(*current_position)
        elif action == 'ES':
            pass
        else:
            print("Ação inválida!")

    # Exibir ambiente após a limpeza
    display_environment(environment)



def oniscient_control(environment):
    dirty_rooms = set(environment.dirty_rooms)

    def find_closest_dirty_room():
        visited = set()
        queue = deque([(environment.current_row, environment.current_col)])

        while queue:
            row, col = queue.popleft()
            if (row, col) in dirty_rooms:
                return row, col

            if (row - 1, col) not in visited and 0 <= row - 1 < environment.num_rows:
                queue.append((row - 1, col))
                visited.add((row - 1, col))
            if (row + 1, col) not in visited and 0 <= row + 1 < environment.num_rows:
                queue.append((row + 1, col))
                visited.add((row + 1, col))
            if (row, col - 1) not in visited and 0 <= col - 1 < environment.num_cols:
                queue.append((row, col - 1))
                visited.add((row, col - 1))
            if (row, col + 1) not in visited and 0 <= col + 1 < environment.num_cols:
                queue.append((row, col + 1))
                visited.add((row, col + 1))

    while dirty_rooms:
        display_environment(environment)

        closest_dirty_room = find_closest_dirty_room()
        if closest_dirty_room is None:
            break

        while (environment.current_row, environment.current_col) != closest_dirty_room:
            if environment.current_row < closest_dirty_room[0]:
                environment.move_down()
            elif environment.current_row > closest_dirty_room[0]:
                environment.move_up()
            elif environment.current_col < closest_dirty_room[1]:
                environment.move_right()
            elif environment.current_col > closest_dirty_room[1]:
                environment.move_left()

            display_environment(environment)

        # Clean the closest dirty room
        environment.clean(*closest_dirty_room)
        dirty_rooms.remove(closest_dirty_room)

    # Exibir ambiente após a limpeza
    display_environment(environment)
    print("Limpeza concluída.")



def main():
    while True:
        while True:
            try:
                num_rows = int(input("Número de linhas: "))
                num_cols = int(input("Número de colunas: "))
                break
            except ValueError:
                print("Entrada inválida. Por favor, insira um número inteiro.")

        dirty_rooms = [(i, j) for i in range(num_rows) for j in range(num_cols) if random.random() < 0.5]  # 50% chance of a room being dirty
        environment = VacuumEnvironment(num_rows, num_cols, dirty_rooms)

        print("\nAmbiente Inicial:")
        display_environment(environment)

        print("\nEscolha o controlador:")
        print("1 - Manual")
        print("2 - Base")
        print("3 - Onisciente")

        choice = input("Escolha o controlador (1/2/3): ")

        if choice == '1':
            manual_control(environment)
        elif choice == '2':
            base_control(environment)
        elif choice == '3':
            oniscient_control(environment)
        else:
            print("Escolha inválida!")

        restart = input("\nDeseja reiniciar o programa (S para sim, qualquer outra tecla para sair)? ")
        if restart.upper() != 'S':
            break

if __name__ == "__main__":
    main()
